In [92]:
import importlib
import numpy as np
import NN
importlib.reload(NN)

<module 'NN' from 'd:\\Work\\Programming\\Jupyter\\Customer Segmentation NN\\NN.py'>

In [93]:
def sigmoid(z):
    return  1 / (1 + np.exp(-z))

In [94]:
layer = NN.DenseLayer(4, 2, sigmoid, None, False, True)

In [95]:
layer.forward_propagation(np.array([[1, 2, 3], [1, 4, 5]])).shape

(2, 4)

In [96]:
layer.calculate_error(np.array([[1, 2, 3], [1, 4, 5]]), np.array([[1], [3]]))

(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)
(1, 2)


array([[ 0.97947198,  0.99956706],
       [-0.83234067,  0.06631503],
       [ 0.89986068,  0.98579282],
       [ 0.10332064, -0.96530818]])

In [97]:
x = [[1, 2, 3], [4, 5, 6]]

In [101]:
np.append([[9, 9, 9]], x, 0)

array([[9, 9, 9],
       [1, 2, 3],
       [4, 5, 6]])